# 02 • EDA of Crime
EDA is an iterative tool that would help us to look into the database and describe it's information.

In this EDA I analyse the data from **crime data/rates**.

## Content
1. Sources selected
2. Describe the data
3. Cleaning data
4. Missing values
5. Analysis and Visualization
6. Additional comments
7. Save clean databases
8. References

## 1. Sources selected

For more info look into the jupyter notebook `01_Datacollection.ipynb` of this repo.

### Crime
TO-DO

## 2. Describe the data

In [113]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [114]:
## To install Altair look the following link
##    https://altair-viz.github.io/getting_started/installation.html
# !pip install altair vega_datasets #<--To install using `pip`
# !conda install -c conda-forge altair vega_datasets #<--To install using `conda`

import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

## 3. Missing values

## 4. Cleaning data


## 5. Analysis and Visualization

## 6. Additional comments



## 7. Save the clean database

## 8. References

Database


EDA
- K. Katari. (Aug 21, 2020).[Exploratory Data Analysis(EDA): Python](https://towardsdatascience.com/exploratory-data-analysis-eda-python-87178e35b14). Towards Data Science.
- J.M. Reid. (Oct 14, 2021). [13 ways to access data in Python](https://towardsdatascience.com/13-ways-to-access-data-in-python-bac5683e0063). Towards Data Science.
- A.K. Garg, V. Cuspinera-Contreras, Y. Qian. (Oct 2020). [Bike Sharing Machine Learning Model, EDA section](https://github.com/vcuspinera/MDS_Bike_Sharing/blob/master/eda/EDA.ipynb).\n",